In [55]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from typing import List

### Constantes

In [2]:
DIR_DATASET_DADOS_BNDES: str = os.path.join('datasets', 'dados-originais')
ARQUIVO_INDICE_ATIVIDADE_ECONOMICA = os.path.join(DIR_DATASET_DADOS_BNDES, '0.1 - Exo - IBC-Br.xls')
ARQUIVO_ESTRUTURA_TERMO_TX_JUROS = os.path.join(DIR_DATASET_DADOS_BNDES, '0.2 - Exo - ETTJ.xls')
ARQUIVO_INDICES_ONIBUS_CAMINHOES = os.path.join(DIR_DATASET_DADOS_BNDES, '1.1 - Endo - FINAME OeC.xls')
ANO_MES_CORRENTE = datetime.today().strftime("%Y-%m")
TAXA_FIXA_TIF = 0.15

### Funções 

In [70]:
def calcula_custo(ano_mes: str, custo_psi: float, spread: float, tx_tjlp: float, tx_ji: float, tx_inflacao: float, tx_di_60: float, tx_selic: float) -> float :
    print(ano_mes)
    print('----')
    # PSI
    if (ano_mes >= '2009-10' and ano_mes <= '2015-12'):
        return custo_psi
    # TJLP
    elif (ano_mes >= '2016-01' and ano_mes <= '2017-12'):
        return spread + tx_tjlp + TAXA_FIXA_TIF
    # TLP
    elif (ano_mes >= '2018-01' and ano_mes <= '2020-07'):
         return ( (1 + (spread + TAXA_FIXA_TIF) / 100) * ( 1 + tx_ji/100) * ( 1 + ETTJ.tx_inflacao/100) - 1) * 100
    # SELIC    
    elif (ano_mes >= '2020-08'):
        return tx_di_60 + tx_selic + TAXA_FIXA_TIF
    else:
         raise ValueError(f"Ano-mes inválido - {ano_mes}" )

### Exógena - Indice de Atividade Econômica

In [3]:
df_indice_atv_economica = pd.read_excel(ARQUIVO_INDICE_ATIVIDADE_ECONOMICA)
df_indice_atv_economica = df_indice_atv_economica[['A_IBC','Ano','Mês']]
df_indice_atv_economica.rename(columns={'A_IBC': 'ind_atv_econ', 'Ano': 'ano', 'Mês': 'mes'}, inplace=True)
df_indice_atv_economica = df_indice_atv_economica.dropna()
df_indice_atv_economica['ano_mes'] = df_indice_atv_economica.ano.map(str) + '-' + df_indice_atv_economica.mes.map("{:02}".format)
df_indice_atv_economica = df_indice_atv_economica[(df_indice_atv_economica.ano_mes >= '2009-09') & (df_indice_atv_economica.ano_mes < ANO_MES_CORRENTE)]


In [4]:
ANO_MES_CORRENTE

'2021-11'

In [5]:
df_indice_atv_economica.head()

,ind_atv_econ,ano,mes,ano_mes
68,129.07,2009,9,2009-09
69,129.74,2009,10,2009-10
70,130.17,2009,11,2009-11
71,131.36,2009,12,2009-12
72,133.46,2010,1,2010-01


---

### Exógena - Estrutura Termo Taxa Juros - Planilha 1 (Plan1 - Analítico) e Planilha 2 (Consolidado)

In [6]:
df_estrutura_tx_juros = pd.read_excel(ARQUIVO_ESTRUTURA_TERMO_TX_JUROS, 'Plan2')
df_estrutura_tx_juros = df_estrutura_tx_juros[['A_DI_12','A_DI_24','A_DI_36', 'A_DI_60', 'A_DI_120', 'Ano', 'Mês']]
df_estrutura_tx_juros.rename(columns={'Ano': 'ano', 'Mês': 'mes'}, inplace=True)
df_estrutura_tx_juros = df_estrutura_tx_juros.dropna()
df_estrutura_tx_juros = df_estrutura_tx_juros.astype({'mes': int, 'ano': int})
df_estrutura_tx_juros['ano_mes'] = df_estrutura_tx_juros.ano.map(str) + '-' + df_estrutura_tx_juros.mes.map("{:02}".format)
df_estrutura_tx_juros = df_estrutura_tx_juros[(df_estrutura_tx_juros.ano_mes >= '2009-09') & (df_estrutura_tx_juros.ano_mes < ANO_MES_CORRENTE)]

In [33]:
df_estrutura_tx_juros_plan1 = pd.read_excel(ARQUIVO_ESTRUTURA_TERMO_TX_JUROS, 'Plan1')
df_estrutura_tx_juros_plan1 = df_estrutura_tx_juros_plan1[['A_Ji','A_TJLP','A_INFLA_FINAME_OEC_MPME','A_INFLA_FINAME_OEC_Grande','Ano', 'Mês']]
df_estrutura_tx_juros_plan1.rename(columns={'Ano': 'ano', 'Mês': 'mes'}, inplace=True)
df_estrutura_tx_juros_plan1 = df_estrutura_tx_juros_plan1.astype({'mes': int, 'ano': int})
df_estrutura_tx_juros_plan1['ano_mes'] = df_estrutura_tx_juros_plan1.ano.map(str) + '-' + df_estrutura_tx_juros_plan1.mes.map("{:02}".format)
df_estrutura_tx_juros_plan1 = df_estrutura_tx_juros_plan1[(df_estrutura_tx_juros_plan1.ano_mes >= '2009-09') & (df_estrutura_tx_juros_plan1.ano_mes < ANO_MES_CORRENTE)]

In [45]:
df_estrutura_tx_juros_plan1.head()

,A_Ji,A_TJLP,A_INFLA_FINAME_OEC_MPME,A_INFLA_FINAME_OEC_Grande,ano,mes,ano_mes
68,NaN,6.0,NaN,NaN,2009,9,2009-09
69,NaN,6.0,5.890385,5.890385,2009,10,2009-10
70,NaN,6.0,5.933936,5.933936,2009,11,2009-11
71,NaN,6.0,6.082671,6.082671,2009,12,2009-12
72,NaN,6.0,6.072903,6.072903,2010,1,2010-01


In [7]:
df_estrutura_tx_juros.head()

,A_DI_12,A_DI_24,A_DI_36,A_DI_60,A_DI_120,ano,mes,ano_mes
68,9.723852,11.376501,12.213310,12.824550,13.203919,2009,9,2009-09
69,9.969168,11.560462,12.282385,12.869093,13.306142,2009,10,2009-10
70,9.984419,11.687233,12.488141,13.121518,13.574884,2009,11,2009-11
71,10.341847,11.944138,12.657941,13.269531,13.789337,2009,12,2009-12
72,10.452661,11.913480,12.576683,13.181880,13.728600,2010,1,2010-01


---

### Endógena - Finame Ônibus e Caminhões

In [8]:
df_finame_oec = pd.read_excel(ARQUIVO_INDICES_ONIBUS_CAMINHOES, 'Plan1')
df_finame_oec = df_finame_oec[['A_DESEM_OEC', 'A_DESEM_OEC_MPME', 'A_DESEM_OEC_Grande', 'A_SPREAD_BNDES_MPME', 'A_SPREAD_BNDES_Grande', 'A_CUSTO_BNDES_MPME','A_CUSTO_BNDES_grande', 'A_VEND_OEC', 'A_SFIXA_SELIC', 'A_APROV_OEC', 'A_APROV_OEC_MPME', 'A_APROV_OEC_Grande', 'Ano', 'Mês']]
df_finame_oec.rename(columns={'Ano': 'ano', 'Mês': 'mes'}, inplace=True)
df_finame_oec.rename(columns={'A_DESEM_OEC': 'desembolso_total', 'A_DESEM_OEC_MPME': 'desembolso_mpme', 'A_DESEM_OEC_Grande': 'desembolso_grande'}, inplace=True)
df_finame_oec.rename(columns={'A_SPREAD_BNDES_MPME': 'spread_mpme', 'A_SPREAD_BNDES_Grande': 'spread_grande', 'A_CUSTO_BNDES_MPME': 'custo_mpme'}, inplace=True)
df_finame_oec.rename(columns={'A_CUSTO_BNDES_grande': 'custo_grande', 'A_VEND_OEC': 'vendas', 'A_SFIXA_SELIC': 'tx_fixa_selic'}, inplace=True)
df_finame_oec.rename(columns={'A_APROV_OEC': 'aprovacao_total', 'A_APROV_OEC_MPME': 'aprovacao_mpme', 'A_APROV_OEC_Grande': 'aprovacao_grande'}, inplace=True)
df_finame_oec['ano_mes'] = df_finame_oec.ano.map(str) + '-' + df_finame_oec.mes.map("{:02}".format)
df_finame_oec = df_finame_oec[(df_finame_oec.ano_mes >= '2009-09') & (df_finame_oec.ano_mes < ANO_MES_CORRENTE)]

In [9]:
df_finame_oec.head()

,desembolso_total,desembolso_mpme,desembolso_grande,spread_mpme,spread_grande,custo_mpme,custo_grande,vendas,tx_fixa_selic,aprovacao_total,aprovacao_mpme,aprovacao_grande,ano,mes,ano_mes
68,1393.82,1045.5,332.6,3.0,3.0,4.0,4.0,12034.0,NaN,1461.208461,1115.629115,345.579346,2009,9,2009-09
69,1767.32,1388.5,349.2,3.0,3.0,4.0,4.0,13123.0,NaN,1817.492460,1437.057832,380.434628,2009,10,2009-10
70,1740.60,1339.9,362.5,3.0,3.0,4.0,4.0,13219.0,NaN,1999.128439,1497.107343,502.021096,2009,11,2009-11
71,1691.54,1259.6,392.2,3.0,3.0,4.0,4.0,15104.0,NaN,2626.210087,1580.255803,1045.954284,2009,12,2009-12
72,2344.65,1682.7,616.7,3.0,3.0,4.0,4.0,11569.0,NaN,2441.408982,1853.163058,588.245924,2010,1,2010-01


### Merge Datasets

In [46]:
df_oec = pd.merge(df_finame_oec, df_indice_atv_economica, how= 'inner',on='ano_mes', suffixes=('','_y'))
df_oec.drop(df_oec.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
df_oec = pd.merge(df_oec, df_estrutura_tx_juros, how= 'inner',on='ano_mes', suffixes=('','_y'))
df_oec.drop(df_oec.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
df_oec = pd.merge(df_oec, df_estrutura_tx_juros_plan1, how= 'inner',on='ano_mes', suffixes=('','_y'))
df_oec.drop(df_oec.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)


In [47]:
df_oec.head()

,desembolso_total,desembolso_mpme,desembolso_grande,spread_mpme,spread_grande,custo_mpme,custo_grande,vendas,tx_fixa_selic,aprovacao_total,...,ind_atv_econ,A_DI_12,A_DI_24,A_DI_36,A_DI_60,A_DI_120,A_Ji,A_TJLP,A_INFLA_FINAME_OEC_MPME,A_INFLA_FINAME_OEC_Grande
0,1393.82,1045.5,332.6,3.0,3.0,4.0,4.0,12034.0,NaN,1461.208461,...,129.07,9.723852,11.376501,12.213310,12.824550,13.203919,NaN,6.0,NaN,NaN
1,1767.32,1388.5,349.2,3.0,3.0,4.0,4.0,13123.0,NaN,1817.492460,...,129.74,9.969168,11.560462,12.282385,12.869093,13.306142,NaN,6.0,5.890385,5.890385
2,1740.60,1339.9,362.5,3.0,3.0,4.0,4.0,13219.0,NaN,1999.128439,...,130.17,9.984419,11.687233,12.488141,13.121518,13.574884,NaN,6.0,5.933936,5.933936
3,1691.54,1259.6,392.2,3.0,3.0,4.0,4.0,15104.0,NaN,2626.210087,...,131.36,10.341847,11.944138,12.657941,13.269531,13.789337,NaN,6.0,6.082671,6.082671
4,2344.65,1682.7,616.7,3.0,3.0,4.0,4.0,11569.0,NaN,2441.408982,...,133.46,10.452661,11.913480,12.576683,13.181880,13.728600,NaN,6.0,6.072903,6.072903


In [48]:
df_oec.columns

Index(['desembolso_total', 'desembolso_mpme', 'desembolso_grande',
       'spread_mpme', 'spread_grande', 'custo_mpme', 'custo_grande', 'vendas',
       'tx_fixa_selic', 'aprovacao_total', 'aprovacao_mpme',
       'aprovacao_grande', 'ano', 'mes', 'ano_mes', 'ind_atv_econ', 'A_DI_12',
       'A_DI_24', 'A_DI_36', 'A_DI_60', 'A_DI_120', 'A_Ji', 'A_TJLP',
       'A_INFLA_FINAME_OEC_MPME', 'A_INFLA_FINAME_OEC_Grande'],
      dtype='object')

---
### MPME

In [49]:
df_oec_mpme = df_oec[['ano', 'mes', 'ano_mes','desembolso_mpme', 'spread_mpme', 'custo_mpme', 
                      'vendas','tx_fixa_selic', 'aprovacao_mpme',  'ind_atv_econ', 
                      'A_Ji', 'A_TJLP','A_INFLA_FINAME_OEC_MPME',
                      'A_DI_12','A_DI_24', 'A_DI_36', 'A_DI_60', 'A_DI_120']]

In [74]:
df_oec_mpme['custo_bndes'] = df_oec_mpme.apply(lambda x: calcula_custo(ano_mes=df_oec_mpme.ano_mes,
                                                                       custo_psi=df_oec_mpme.custo_mpme,
                                                                       spread=df_oec_mpme.spread_mpme,
                                                                       tx_tjlp=df_oec_mpme.A_TJLP,
                                                                       tx_ji=df_oec_mpme.A_Ji,
                                                                       tx_inflacao=df_oec_mpme.A_INFLA_FINAME_OEC_MPME,
                                                                       tx_di_60=df_oec_mpme.A_DI_60,
                                                                       tx_selic=df_oec_mpme.tx_fixa_selic), 
                                               axis=1)

0      2009-09
1      2009-10
2      2009-11
3      2009-12
4      2010-01
        ...   
141    2021-06
142    2021-07
143    2021-08
144    2021-09
145    2021-10
Name: ano_mes, Length: 146, dtype: object
----


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().